In [1]:
#default_exp rss_feeds 

In [2]:
#export
from operator import itemgetter

import numpy as np
import tqdm
import bs4
import feedparser
import seaborn as sns
import pandas as pd

In [3]:
%cd ..

/home/kuba/Projects/pytorch_hackathon


In [4]:
pd.set_option("display.max_colwidth", 250)

# RSS Feeds


In [5]:
#export
medium_publications = [
    'the-artificial-impostor',
    'pytorch',
    'dair.ai',
    'towards-artificial-intelligence',
    'swlh',
    '@ODSC',
    'doctrine',
    'paperswithcode'
]


medium_url_template = 'https://medium.com/feed/{}'
medium_url = medium_url_template.format(medium_publications[0])
medium_urls = [medium_url_template.format(publication) for publication in medium_publications]

In [6]:
#export
subreddits = [
    'MachineLearning',
    'deeplearning',
    'datascience',
    'cognitivelinguistics',
    'TopOfArxivSanity',
    'kaggle'
]

reddit_url_template = 'https://www.reddit.com/r/{}/.rss'
reddit_url = reddit_url_template.format(subreddits[0])
reddit_urls = [reddit_url_template.format(subreddit) for subreddit in subreddits]

In [7]:
#export


def get_article_text(article):
    article_html_content = article['content'][0]['value']
    article_text = bs4.BeautifulSoup(article_html_content).text
    return article_text


def get_feed_article_texts(feed):
    return [get_article_text(article) for article in feed['entries'] if 'content' in article.keys()]


def get_feed_article_df(feed):
    feed_df = pd.DataFrame.from_records(feed['entries'])
    feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)
    return feed_df


def add_field(df, field_name, values):
    df[field_name] = values
    return df

In [8]:
#export
paperswithcode_url = 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest' 
hackernews_url = 'https://news.ycombinator.com/rss' 
rss_feed_urls = [paperswithcode_url, hackernews_url] + medium_urls + reddit_urls

In [9]:
feed_urls = pd.read_table('data/feeds.txt', header=None).iloc[:,0].values

In [10]:
for url in rss_feed_urls:
    print(url)

https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest
https://news.ycombinator.com/rss
https://medium.com/feed/the-artificial-impostor
https://medium.com/feed/pytorch
https://medium.com/feed/dair.ai
https://medium.com/feed/towards-artificial-intelligence
https://medium.com/feed/swlh
https://medium.com/feed/@ODSC
https://medium.com/feed/doctrine
https://medium.com/feed/paperswithcode
https://www.reddit.com/r/MachineLearning/.rss
https://www.reddit.com/r/deeplearning/.rss
https://www.reddit.com/r/datascience/.rss
https://www.reddit.com/r/cognitivelinguistics/.rss
https://www.reddit.com/r/TopOfArxivSanity/.rss
https://www.reddit.com/r/kaggle/.rss


### Loading RSS feed articles to a dataframe

In [11]:
#export
from datetime import date

def to_date(x):
    if str(x)[0].isdigit():
        return str(x)[:10]
    else:
        return str(x).split(' ')[1] + ' ' + str(x).split(' ')[2] + ' ' + str(x).split(' ')[3]


def get_feed_df(feed_urls):
    feeds = [
        (feed_url, feedparser.parse(feed_url))
        for feed_url in tqdm.tqdm(feed_urls)
    ]
    df = pd.concat(
        [
            add_field(get_feed_article_df(feed), 'feed', feed_url)
            for (feed_url, feed) in feeds
            if len(feed['entries']) > 0
        ]
    )
    
    df['date'] = df['updated']
    df['date'] = df['date'].fillna(df['published'])
    df['date'] = df['date'].fillna(date.today())
    
    df['date'] = df['date'].apply(lambda x: to_date(x))
    col = df['date'].unique()
    
    #return col
    return df
    

100%|██████████| 16/16 [00:07<00:00,  2.09it/s]


In [12]:
feed_df.shape

(296, 23)

In [13]:
feed_df

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail,date
0,Pose2Mesh: Graph Convolutional Network for 3D Human Pose and Mesh Recovery from a 2D Human Pose,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Pose2Mesh: Graph Convolutional Network for 3D Human Pose and Mesh Recovery from a 2D Human Pose'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/pose2mesh-graph-convolutional-network-for-3d'}]",https://paperswithcode.com/paper/pose2mesh-graph-convolutional-network-for-3d,"Most of the recent deep learning-based 3D human pose and mesh estimation methods regress the pose and shape parameters of human mesh models, such as SMPL and MANO, from an input image. <p>Code: <a href=""https://github.com/hongsukchoi/Pose2Mesh_RE...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Most of the recent deep learning-based 3D human pose and mesh estimation methods regress the pose and shape parameters of human m...",https://paperswithcode.com/paper/pose2mesh-graph-convolutional-network-for-3d,False,NaN,"Most of the recent deep learning-based 3D human pose and mesh estimation methods regress the pose and shape parameters of human mesh models, such as SMPL and MANO, from an input image. Code: https://github.com/hongsukchoi/Pose2Mesh_RELEASE",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-25
1,Uncertainty Estimation in Medical Image Denoising with Bayesian Deep Image Prior,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Uncertainty Estimation in Medical Image Denoising with Bayesian Deep Image Prior'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/uncertainty-estimation-in-medical-image'}]",https://paperswithcode.com/paper/uncertainty-estimation-in-medical-image,"We use a randomly initialized convolutional network as parameterization of the reconstructed image and perform gradient descent to match the observation, which is known as deep image prior. <p>Code: <a href=""https://github.com/mlaves/uncertainty-...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'We use a randomly initialized convolutional network as parameterization of the reconstructed image and perform gradient descent t...",https://paperswithcode.com/paper/uncertainty-estimation-in-medical-image,False,"[{'term': 'Image denoising', 'scheme': None, 'label': None}]","We use a randomly initialized convolutional network as parameterization of the reconstructed image and perform gradient descent to match the observation, which is known as deep image prior. Code: https://github.com/mlaves/uncertainty-deep-image-p...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-25
2,Yet Another Intermediate-Level Attack,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Yet Another Intermediate-Level Attack'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/yet-another-intermediate-level-attack'}]",https://paperswithcode.com/paper/yet-another-intermediate-level-attack,"The transferability of adversarial examples across deep neural network (DNN) models is the crux of a spectrum of black-box attacks. <p>Code: <a href=""https://github.com/qizhangli/ila-plus-plus"">https://github.com/qizhangli/ila-plus-plus</a></p>","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'The transferability of adversarial examples across deep neural network (DNN) models is the crux of a spectrum of black-box attack...",https://paperswithcode.com/paper/yet-another-intermediate-level-attac

In [14]:
pd.to_datetime(feed_df['date'])

0    2020-08-25
1    2020-08-25
2    2020-08-25
3    2020-08-25
4    2020-08-25
        ...    
20   2020-07-23
21   2020-07-21
22   2020-07-19
23   2020-07-18
24   2020-07-17
Name: date, Length: 296, dtype: datetime64[ns]